# Approximate betweenness centrality using neural networks
Here we start to approximate the betweennesss centrality using neural networks over a peer-2-peer network Gnutella. Gnutella is a set of datasets consisting of 9 networks ranging from 6,300 to 63,000 nodes. Our goal is to train a neural network on the smallest Gnurella graph and evaluate it on a much larger graph. We will guide you through this step by step.

You can find Gnutella datasets at http://snap.stanford.edu/data/index.html. We will use p2p-Gnutella08 for training and p2p-Gnutella04 for testing.

Note:
1. Copy this notebook to your Google drive in order to execute it.
2. Make sure to upload the data files in HW4 to your google drive and to modify their corresponding directories in the code.

# Part 1: Training a model on Gnutella 08

## Preprocessing Gnutella08 dataset


In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import networkx as nx
import scipy

2026-02-04 19:47:23.887086: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Parameters

# choose an embedding size for Structure2Vec
EMBED_SIZE = 256

# choose number of dense layers in the neural network
NUM_LAYERS = 2

# choose number of folds for cross validation
NUM_FOLD = 5

# choose number of epochs for training
NUM_EPOCHS = 5

In [ ]:
# Normalize a list of values
# NO NEED TO CHANGE

def _normalize_array_by_rank(true_value, nr_nodes):
  # true_value is a list of values you want to normalize and nr_nodes is the number of nodes in the list

  rank = np.argsort(true_value, kind='mergesort', axis=None) #deg list get's normalised
  norm = np.empty([nr_nodes])

  for i in range(0, nr_nodes):
    norm[rank[i]] = float(i+1) / float(nr_nodes)

  max = np.amax(norm)
  min = np.amin(norm)
  if max > 0.0 and max > min:
    for i in range(0, nr_nodes):
      norm[i] = 2.0*(float(norm[i] - min) / float(max - min)) - 1.0
  else:
    print("Max value = 0")

  return norm, rank

In [6]:
#Read in and create NetworkX Graph; G

#TO-DO: The path needs to be changed according to your dataset directory in your GOOGLE DRIVE
path = './data/p2p-Gnutella08.txt'

G = nx.read_edgelist(path, comments='#', delimiter=None, create_using=nx.DiGraph,
                  nodetype=None, data=True, edgetype=None, encoding='utf-8')

#print(nx.info(G))

In [7]:
# Creating list of Degrees of the nodes in G and normalising them:

deg_lst = [val for (node, val) in G.degree()]
nr_nodes = G.number_of_nodes()
print("deg_lst: \n", deg_lst)

degree_norm, degree_rank = _normalize_array_by_rank(deg_lst, nr_nodes)

deg_lst: 
 [10, 1, 6, 87, 69, 83, 1, 60, 84, 59, 7, 15, 4, 7, 17, 12, 2, 15, 1, 3, 2, 79, 2, 11, 11, 2, 13, 7, 6, 1, 11, 54, 95, 50, 80, 70, 90, 91, 80, 92, 19, 90, 75, 79, 23, 52, 30, 5, 5, 7, 17, 23, 12, 16, 21, 14, 1, 2, 9, 87, 76, 21, 67, 48, 91, 56, 14, 1, 6, 6, 16, 2, 39, 4, 1, 17, 20, 2, 4, 97, 71, 81, 94, 91, 40, 8, 10, 14, 6, 6, 5, 12, 4, 2, 67, 57, 69, 11, 1, 4, 12, 15, 5, 6, 3, 4, 13, 2, 1, 13, 2, 10, 15, 2, 3, 1, 12, 5, 9, 14, 37, 15, 25, 4, 5, 13, 3, 12, 2, 6, 5, 2, 7, 13, 5, 4, 19, 7, 16, 83, 72, 19, 14, 8, 6, 16, 20, 5, 3, 2, 2, 14, 20, 12, 12, 11, 11, 11, 12, 2, 2, 13, 1, 30, 12, 1, 5, 2, 16, 11, 9, 5, 15, 3, 1, 3, 4, 7, 4, 15, 17, 7, 2, 18, 1, 8, 15, 4, 6, 13, 3, 10, 2, 12, 3, 7, 10, 4, 1, 3, 3, 17, 7, 5, 1, 5, 4, 14, 7, 6, 3, 2, 4, 4, 11, 2, 12, 10, 11, 5, 4, 12, 11, 12, 20, 15, 14, 2, 1, 5, 14, 12, 14, 13, 3, 14, 12, 12, 3, 27, 2, 11, 2, 2, 14, 9, 6, 3, 13, 1, 1, 5, 1, 1, 4, 7, 1, 2, 2, 9, 12, 4, 4, 5, 17, 9, 14, 25, 13, 4, 2, 5, 7, 11, 4, 5, 4, 4, 1, 10, 12, 12, 5, 

In [8]:
# Computing Ground-truth values and normalising them:

b = [v for v in nx.betweenness_centrality(G).values()]

BC_norm_cent, BC_cent_rank = _normalize_array_by_rank(b, nr_nodes)

In [9]:
# Define Structure2Vec
# NO NEED TO CHANGE

def Structure2Vec(G, nr_nodes, degree_norm, num_features=1, embed_size=512, layers=2):

  #build feature matrix
  def get_degree(i):
    return degree_norm[i]

  def build_feature_matrix():
    n = nr_nodes
    feature_matrix = []
    for i in range(0, n):
      feature_matrix.append(get_degree(i))
    return feature_matrix
  #Structure2Vec node embedding
  A = nx.to_numpy_array(G)

  dim = [nr_nodes, num_features]


  node_features = tf.cast(build_feature_matrix(), tf.float32)
  node_features = tf.reshape(node_features, dim)

  initializer = tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0,
                                                                mode="fan_avg",
                                                                distribution="uniform")
  #print(initializer)

  A = tf.sparse.from_dense(A)
  A = tf.cast(A, tf.float32)
  w1 = tf.Variable(initializer((num_features, embed_size)), trainable=True,
                                  dtype=tf.float32, name="w1")
  w2 = tf.Variable(initializer((embed_size, embed_size)), trainable=True,
                                  dtype=tf.float32, name="w2")
  w3 = tf.Variable(initializer((1,embed_size)), trainable=True, dtype=tf.float32, name="w3")
  w4 = tf.Variable(initializer([]), trainable=True, dtype=tf.float32, name="w4")

  wx_all = tf.matmul(node_features, w1)  # NxE

  #computing X1:
  #sparse.reduce_sum: Computes the sum of elements across dimensions of a SparseTensor.
  weight_sum_init = tf.sparse.reduce_sum(A, axis=1, keepdims=True, ) #takes adjacency matrix
  n_nodes = tf.shape(input=A)[1]

  weight_sum = tf.multiply(weight_sum_init, w4)
  weight_sum = tf.nn.relu(weight_sum)  # Nx1
  weight_sum = tf.matmul(weight_sum, w3)  # NxE

  weight_wx = tf.add(wx_all, weight_sum)
  current_mu = tf.nn.relu(weight_wx)  # NxE = H^0

  for i in range(0, layers):
    neighbor_sum = tf.sparse.sparse_dense_matmul(A, current_mu)
    neighbor_linear = tf.matmul(neighbor_sum, w2)  # NxE

    current_mu = tf.nn.relu(tf.add(neighbor_linear, weight_wx))  # NxE

  mu_all = current_mu

  return mu_all

In [10]:
# Converting the graph structure into vectors

mu_all = Structure2Vec(G, nr_nodes, degree_norm, embed_size=EMBED_SIZE)

I0000 00:00:1770263295.450779 2673770 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7381 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1
2026-02-04 19:48:15.614226: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 317620808 exceeds 10% of free system memory.


## Training a Neural Network

In [11]:
# Building NN model

UNITS = int(EMBED_SIZE/2)
def build_model():
  model = tf.keras.Sequential()
  model.add(tf.keras.Input(shape=(EMBED_SIZE,)))

  # choose the number of layers to construct your network
  for _ in range(NUM_LAYERS):
    model.add(tf.keras.layers.Dense(UNITS, activation ="relu"))

  model.add(tf.keras.layers.Dense(1))
  model.compile(optimizer='sgd', loss='mse')

  model.summary()

  return model

model = build_model()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,537 (193.50 KB)

 Trainable params: 49,537 (193.50 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Construct training set and groundtruth

x_train = mu_all
y_train = BC_norm_cent
print(tf.shape(x_train))
print(tf.shape(y_train))

tf.Tensor([6301  256], shape=(2,), dtype=int32)
tf.Tensor([6301], shape=(1,), dtype=int32)


In [13]:
# Computing cross validation
# NO NEED TO CHANGE
all_scores = []
k = NUM_FOLD
num_val_samples = len(x_train) // k
for i in range(k):
  print('processing fold #', i)
  val_data = x_train[i*num_val_samples: (i+1) * num_val_samples]
  val_targets = y_train[i*num_val_samples: (i+1)*num_val_samples]

  partial_train_data = np.concatenate(
      [x_train[:i*num_val_samples],
      x_train[(i+1)*num_val_samples:]],
      axis = 0)
  print(tf.shape(partial_train_data))

  partial_train_targets = np.concatenate(
      [y_train[:i*num_val_samples],
      y_train[(i+1)*num_val_samples:]],
      axis = 0)

  # Training
  callbacks =  tf.keras.callbacks.EarlyStopping(
      monitor= 'loss', min_delta=0, patience=3, verbose=1,
      mode='auto', baseline=None, restore_best_weights=False)

  model.fit(partial_train_data, partial_train_targets,
            epochs = NUM_EPOCHS, batch_size = 1, callbacks = callbacks, verbose = 1)
  print("model.metrics_names: ", model.metrics_names)

  val_loss = model.evaluate(val_data, val_targets, verbose = 1)

  all_scores.append(val_loss)
  print(all_scores)

processing fold # 0
tf.Tensor([5041  256], shape=(2,), dtype=int32)
Epoch 1/5


2026-02-04 19:48:16.684692: I external/local_xla/xla/service/service.cc:163] XLA service 0x7fa970005040 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2026-02-04 19:48:16.684703: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce GTX 1080, Compute Capability 6.1
2026-02-04 19:48:16.703326: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2026-02-04 19:48:16.732864: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91900


 185/5041 ━━━━━━━━━━━━━━━━━━━━ 3s 821us/step - loss: 0.2709 

I0000 00:00:1770263296.888119 2674657 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5041/5041 ━━━━━━━━━━━━━━━━━━━━ 4s 786us/step - loss: 0.0956
Epoch 2/5
5041/5041 ━━━━━━━━━━━━━━━━━━━━ 4s 792us/step - loss: 0.0704
Epoch 3/5
5041/5041 ━━━━━━━━━━━━━━━━━━━━ 4s 787us/step - loss: 0.0640
Epoch 4/5
5041/5041 ━━━━━━━━━━━━━━━━━━━━ 4s 788us/step - loss: 0.0605
Epoch 5/5
5041/5041 ━━━━━━━━━━━━━━━━━━━━ 4s 791us/step - loss: 0.0580
model.metrics_names:  ['loss']
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2033 
[0.2033482939004898]
processing fold # 1
tf.Tensor([5041  256], shape=(2,), dtype=int32)
Epoch 1/5
5041/5041 ━━━━━━━━━━━━━━━━━━━━ 4s 793us/step - loss: 0.0837
Epoch 2/5
5041/5041 ━━━━━━━━━━━━━━━━━━━━ 4s 790us/step - loss: 0.0821
Epoch 3/5
5041/5041 ━━━━━━━━━━━━━━━━━━━━ 4s 793us/step - loss: 0.0795
Epoch 4/5
5041/5041 ━━━━━━━━━━━━━━━━━━━━ 4s 827us/step - loss: 0.0786
Epoch 5/5
5041/5041 ━━━━━━━━━━━━━━━━━━━━ 4s 785us/step - loss: 0.0785
model.metrics_names:  ['loss']
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0836 
[0.2033482939004898, 0.08358737081289291]
process

In [14]:
# Computing Kendall on trained set

x_new = x_train
y_pred = model.predict(x_new)

# compute kendalltau using the prediction results and the groundtruth
from scipy import stats
kendall_tau, p_value = scipy.stats.kendalltau(BC_norm_cent,y_pred)

197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [15]:
# Print your kendalltau score
# Make sure your kendalltau score is at least 0.70
# PRINT HERE
print(kendall_tau)

0.7299190148642852


In [16]:
# You could save this model for part 2

model.save("./GN08_model_plain.h5")

# Part 2: Evaluating the trained model on Gnutella 04

Hints:
1. Write down the evaluation using the functions and codes in Part 1
2. Compute the groundtruth of betweenness centrality using NetworkX could take around 1 hour. Keep your Colab opened and be patient.

In [18]:
'''Gnutella 04'''
# change the path to your own directory
path2 = './data/p2p-Gnutella04.txt'

G2 = nx.read_edgelist(path2, comments='#', delimiter=None, create_using=nx.DiGraph,
                  nodetype=None, data=True, edgetype=None, encoding='utf-8')

#print(nx.info(G2))

In [19]:
deg_lst2 = [val for (node, val) in G2.degree()]
nr_nodes2 = G2.number_of_nodes()
degree_norm2, degree_rank2 = _normalize_array_by_rank(deg_lst2, nr_nodes2)

In [20]:
print("Computing betweenness centrality for G2")
b2 = [v for v in nx.betweenness_centrality(G2).values()]
BC_norm_cent2, BC_cent_rank2 = _normalize_array_by_rank(b2, nr_nodes2)

Computing betweenness centrality for G2


In [21]:
mu_all2 = Structure2Vec(G2, nr_nodes2, degree_norm2, embed_size=EMBED_SIZE)

2026-02-04 19:57:45.815480: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 946299008 exceeds 10% of free system memory.


In [22]:
# model = tf.keras.models.load_model("./GN08_model_plain.h5")

In [23]:
x_test = mu_all2
y_pred2 = model.predict(x_test)

# Flatten predictions
if len(y_pred2.shape) > 1:
    y_pred2 = y_pred2.flatten()

print(f"Predictions shape: {y_pred2.shape}")
print(f"Ground truth shape: {len(BC_norm_cent2)}")

340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 866us/step
Predictions shape: (10876,)
Ground truth shape: 10876


In [24]:
kendall_tau2, p_value2 = scipy.stats.kendalltau(BC_norm_cent2, y_pred2)

print(f"Kendall tau correlation on Gnutella 04: {kendall_tau2}")
print(f"P-value: {p_value2}")

Kendall tau correlation on Gnutella 04: -0.4105517022447545
P-value: 0.0
